In [1]:
import os

import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import sklearn
import tensorflow as tf
import tensorflow_addons as tfa
from PIL import Image
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    classification_report,
    f1_score,
    plot_confusion_matrix,
    precision_recall_curve,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import (
    StratifiedKFold,
    StratifiedShuffleSplit,
    train_test_split,
)
from tensorflow import keras

from tensorflow.keras.applications.efficientnet import EfficientNetB4, EfficientNetB7
from tensorflow.keras.layers import (
    AveragePooling2D,
    AvgPool2D,
    Conv2D,
    Dense,
    Flatten,
    GlobalAveragePooling2D,
    MaxPooling2D,
)
from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import notebook, tqdm

In [2]:
PATH = "/app/_data/"
BATCH_SIZE = 32
SEED = 1488
IMAGE_SIZE = (380, 380)
NUM_CLASSES = 6
INPUT_SHAPE = (380, 380, 3)
TRAIN_IMG_PATH = "/app/_data/train_images/"
TEST_IMG_PATH = "/app/_data/test_images/"
# MODEL_PATH = '/app/_data/models/effnet7_2_6cl_frozen_40ep_42.h5'

In [3]:
labels = pd.read_csv(PATH + "train_labels.csv")

In [4]:
sample_submission = pd.read_csv(PATH + "sample_submission.csv")

In [5]:
labels = labels.join(labels["labels"].str.get_dummies(sep=" "))

In [6]:
labels.head(1)

,image,labels,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab
0,800113bb65efe69e.jpg,healthy,0,0,1,0,0,0


In [7]:
feature_columns = labels.columns[2:].tolist()
feature_columns

['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']

In [8]:
feature_columns

['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']

In [36]:
os.listdir(PATH)

['train_images',
 'images',
 'train_0.csv',
 'models',
 'train_labels.csv',
 'cache',
 '380',
 'list_to_drop.csv',
 'duplicates_08.csv',
 'plant-pathology-2020-fgvc7.zip',
 'test_images',
 'sample_submission.csv',
 'train.csv',
 'duplicates_09.csv',
 'duplicates_only_08.csv',
 '.ipynb_checkpoints']

In [39]:
labels_20 = pd.read_csv(PATH+'train_20.csv')
labels_20

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0
...,...,...,...,...,...
1816,Train_1816,0,0,0,1
1817,Train_1817,1,0,0,0
1818,Train_1818,1,0,0,0
1819,Train_1819,0,0,1,0


### StratifiedKFold

In [9]:
def pred_to_labels(pred, thresh=0.5, labels=feature_columns):

    pred = [labels[i] for i in range(len(labels)) if pred[i] > thresh]
    pred = ' '.join(pred)
    return pred

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
data_augmentation = keras.Sequential(
    [
        keras.layers.experimental.preprocessing.RandomFlip(
            "horizontal_and_vertical", seed=SEED
        ),
        keras.layers.experimental.preprocessing.RandomRotation(0.15, seed=SEED),
        keras.layers.experimental.preprocessing.RandomZoom(0.25, 0.25, seed=SEED),
        keras.layers.experimental.preprocessing.RandomContrast(factor=0.05, seed=SEED),
        keras.layers.experimental.preprocessing.RandomTranslation(
            height_factor=0.2, width_factor=0.2, seed=SEED
        ),
    ]
)

In [11]:
# def write_np(file_path):
#     img = tf.io.read_file(TRAIN_IMG_PATH + file_path)
#     img = tf.image.decode_jpeg(img, channels=3)
#     img = tf.image.convert_image_dtype(img, tf.float32)
#     img = tf.image.resize(
#         img,
#         IMAGE_SIZE,
#     )
#     img = tf.image.convert_image_dtype(img, tf.uint8)
#     img = Image.fromarray(np.array(img))
#     img.save(PATH + '380/'+file_path)
#     return img
#     np.save(PATH + 'cache/'+file_path[:-4], img)
# labels['image'].apply(write_np)

In [42]:
# def write_np(file_path):
#     img = tf.io.read_file(PATH+'images/' + file_path+'.jpg')
#     img = tf.image.decode_jpeg(img, channels=3)
#     img = tf.image.convert_image_dtype(img, tf.float32)
#     img = tf.image.resize(
#         img,
#         IMAGE_SIZE,
#     )
#     img = tf.image.convert_image_dtype(img, tf.uint8)
#     img = Image.fromarray(np.array(img))
#     img.save(PATH + '380/'+file_path+'.jpg')
#     return img
# #     np.save(PATH + 'cache/'+file_path[:-4], img)
# labels_20['image_id'].apply(write_np)

0       <PIL.Image.Image image mode=RGB size=380x380 a...
1       <PIL.Image.Image image mode=RGB size=380x380 a...
2       <PIL.Image.Image image mode=RGB size=380x380 a...
3       <PIL.Image.Image image mode=RGB size=380x380 a...
4       <PIL.Image.Image image mode=RGB size=380x380 a...
                              ...                        
1816    <PIL.Image.Image image mode=RGB size=380x380 a...
1817    <PIL.Image.Image image mode=RGB size=380x380 a...
1818    <PIL.Image.Image image mode=RGB size=380x380 a...
1819    <PIL.Image.Image image mode=RGB size=380x380 a...
1820    <PIL.Image.Image image mode=RGB size=380x380 a...
Name: image_id, Length: 1821, dtype: object

In [48]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
X = labels["image"]
y = labels[feature_columns]
for train_index, valid_index in sss.split(labels["image"], labels['labels']):
    train, valid = labels.loc[train_index], labels.loc[valid_index]

In [44]:
# from 380 uint8
def parse_image2(file_path):
    img = tf.io.read_file(PATH+'380/'+file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img

In [45]:
# v1 from orig
def parse_image(file_path):
    img = tf.io.read_file(TRAIN_IMG_PATH+file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(
        img,
        IMAGE_SIZE,
    )
    img = tf.image.convert_image_dtype(img, tf.uint8)
    return img


In [46]:
def prepare_dataset(df, augmentation=False):
    dataset = tf.data.Dataset.from_tensor_slices(
        (df["image"].values, df[feature_columns].astype('uint8').values)
    )
    dataset = dataset.map(lambda x, y: (parse_image2(x), y))
    dataset = dataset.batch(BATCH_SIZE)

    if augmentation:
        dataset = dataset.map(
            lambda x, y: (data_augmentation(x, training=True), y),
            num_parallel_calls=AUTOTUNE,
        )
    dataset = dataset.repeat().prefetch(buffer_size=AUTOTUNE)
    return dataset

In [49]:
ds_train = prepare_dataset(train, augmentation = True)
ds_valid = prepare_dataset(valid)

In [16]:
def parse_test_image(img_name):
    img = tf.io.read_file(img_name)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(
        img,
        IMAGE_SIZE,
    )
    img = tf.image.convert_image_dtype(img, tf.uint8)
    return img


def predict_new(path, model):
    img = parse_test_image(path)
    img = tf.expand_dims(img,axis = 0)
    pred = model.predict(img)
    return pred_to_labels(pred[0])

In [17]:
policy = keras.mixed_precision.experimental.Policy('mixed_float16')
keras.mixed_precision.experimental.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Quadro RTX 5000, compute capability 7.5


In [18]:
def scheduler(epoch, lr):
    if epoch < 5:
        return 0.01
    elif 5 <= epoch <10:
        return 0.001
    elif 10 <= epoch <50:
        return 0.0005
    else:
        return 0.0001

In [19]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=15, restore_best_weights=True, verbose=0, mode='min',
    ),
    keras.callbacks.LearningRateScheduler(schedule = scheduler, verbose=1)
]

## StratifiedKFold

In [21]:
# inputs = keras.Input(shape=INPUT_SHAPE)
# base_model = keras.applications.EfficientNetB4(weights=None, include_top=False)
# base_model.load_weights('/app/_data/models/efficientnet-b4_noisy-student_notop.h5', by_name=True, skip_mismatch = True)
# x = base_model(inputs)
# x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
# x = keras.layers.Flatten(name="flatten")(x)
# outputs = keras.layers.Dense(NUM_CLASSES, activation="sigmoid")(x)
# effnet = keras.Model(inputs=inputs, outputs=outputs)


In [20]:
# skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)
# X = labels["image"]
# y = labels[feature_columns]
# for i, (train_index, valid_index) in enumerate(skf.split(labels["image"], labels['labels'])):
#     train, valid = labels.loc[train_index], labels.loc[valid_index]
#     model_name = 'eff4_ns_kf_' + str(i+1) + '.h5'
#     ds_train = prepare_dataset(train, augmentation = True)
#     ds_valid = prepare_dataset(valid)
#     model = effnet
#     model.compile(
#     loss="binary_crossentropy",
#     optimizer=Adam(),
#     metrics=['acc',
#         keras.metrics.Recall(),
#         keras.metrics.Precision(),
#         tfa.metrics.F1Score(num_classes=NUM_CLASSES, average="micro"),
#     ],
# )
#     history = model.fit(
#     ds_train,
#     validation_data=ds_valid,
#     epochs=100,
#     steps_per_epoch=(train.shape[0]*0.8)//BATCH_SIZE, 
#     validation_steps= (valid.shape[0]*0.2)//BATCH_SIZE,
#     verbose=1,
#     use_multiprocessing=True,
#     callbacks = callbacks
# )
#     model.save("/app/_data/models/"+model_name)
    

In [21]:
def plot_curves(model, loss=False):
    df = pd.DataFrame(model.history)
    if loss:
        plt.figure(figsize=(12, 6))
        plt.title("loss")
        plt.plot(df.iloc[:, 0], label="loss")
        plt.plot(df.iloc[:, 5], label="val_loss")
        plt.legend()
        plt.show()
    else:
        plt.figure(figsize=(12, 20))
        plt.subplot(4, 1, 1)
        plt.title("accuracy")
        plt.plot(df.iloc[:, 1], label="accuracy")
        plt.plot(df.iloc[:, 6], label="val_accuracy")
        plt.legend()

        plt.subplot(4, 1, 2)
        plt.title("recall")
        plt.plot(df.iloc[:, 2], label="recall")
        plt.plot(df.iloc[:, 7], label="val_recall")
        plt.legend()

        plt.subplot(4, 1, 3)
        plt.title("precision")
        plt.plot(df.iloc[:, 3], label="precision")
        plt.plot(df.iloc[:, 8], label="val_precision")
        plt.legend()

        plt.subplot(4, 1, 4)
        plt.title("f1")
        plt.plot(df.iloc[:, 4], label="f1")
        plt.plot(df.iloc[:, 9], label="val_f1")
        plt.legend()
        plt.show();

In [22]:
# plot_curves(history,loss=True)

## predictions for all

In [23]:
# model = keras.models.load_model('/app/_data/models/eff4_ns_kf_1.h5')

# def parse_all(file_path):
#     img = tf.io.read_file(PATH+'380/'+file_path)
#     img = tf.image.decode_jpeg(img, channels=3)
#     return img

# def predict_new(path, model):
#     img = parse_all(path)
#     img = tf.expand_dims(img,axis = 0)
#     pred = model.predict(img)
#     return pred_to_labels(pred[0])

# df_sub = pd.DataFrame(columns=['image','labels'])
# for img_name in os.listdir(PATH+'380/'):
#     pred = predict_new(img_name, model)
#     df_sub = df_sub.append( {'image': img_name, 'labels': pred}, ignore_index = True )
# print(df_sub.head())

In [49]:
# df_sub = df_sub.merge(labels[['image', 'labels']], on='image', how='left', suffixes=('_pred', '_true'))
# df_sub.to_csv('/app/sandbox/wrong_predictions/prediction_raw_8_5.csv')

## model weights

In [56]:
# all_weights = []
# for mod_path in [MODEL_PATH1,MODEL_PATH2,MODEL_PATH3,MODEL_PATH4,MODEL_PATH5]:
#     model = keras.models.load_model(mod_path)
#     all_weights.append(model.get_weights())
# all_weights_mean = np.mean(np.array(all_weights), axis = 0)
# w1 = np.array(all_weights[2])
# w1[-10:] = all_weights_mean[-10:]
# model.set_weights(w1)
# model.evaluate(ds_valid, steps = (valid.shape[0]*0.2)//BATCH_SIZE)